In [41]:
# 필요한 라이브러리 정의
#!pip install gradio
import gradio as gr
#!pip install gtts
import gtts
#!pip install ultralytics
from ultralytics import YOLO
import torch
import cv2
import numpy as np
from tts_final import tts
import sys
import os
from collections import Counter

# tts_final 모듈 경로 추가
sys.path.append('/content')





In [42]:
# 영어로 설정된 class를 한국어로 번역 (tts를 위해서)
from collections import Counter

def wordChanger(word):
    if word == "can":
        return "캔"
    elif word == "glass":
        return "유리"
    elif word == "paper":
        return "종이"
    elif word == "pet":
        return "페트병"
    elif word == "plastic":
        return "플라스틱"
    elif word == "styrofoam":
        return "스티로폼"
    elif word == "vinyl":
        return "비닐"

In [43]:
# YOLO 모델 로드
model = YOLO("/content/best.pt")

In [47]:
# 객체 감지 함수 구현
def objDetect(image_path):
    # 이미지 경로에서 객체 감지 실행
    results = model.predict(image_path)

    # 결과에서 객체 이름 추출
    detected_objects = []
    for result in results:
        detected_objects.extend([result.names[int(cls)] for cls in result.boxes.cls])

    # 객체의 개수 세기
    object_counts = Counter(detected_objects)
    object_counts_dict = dict(object_counts)

    # 최빈값 찾기
    most_common_word, _ = object_counts.most_common(1)[0]

    # 최빈값 번역하기
    best = wordChanger(most_common_word)

    return best
# 이미지에서 객체를 감지하고 TTS로 변환하는 함수
def process_image(image):
    detected_objects = objDetect(image)
    tts_result = tts(detected_objects)
    return tts_result


In [51]:
# Gradio 인터페이스 생성
with gr.Blocks(css="styles.css") as iface:
    # 타이틀 및 설명 텍스트 추가
    gr.Markdown("<div class='container'><h1>Voicena 보이시네</h1>")
    gr.Markdown("사진을 등록해주세요.</div>")

    with gr.Column():
        input_image = gr.Image(type="pil", label="Input Image")
        submit_button = gr.Button("Process Image")
        output_audio = gr.Audio(label="TTS Result", type="filepath", autoplay=True)

    submit_button.click(fn=process_image, inputs=input_image, outputs=output_audio)

    # 하단바 추가
    gr.Markdown("<div class='container'><hr>")
    gr.Markdown("© 2024 8 color bird. All rights reserved.</div>")

# 인터페이스 실행 및 공유 URL 생성
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://28b3be9e617c5ea02e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
